In [ ]:
import os
import qp
import jax
import json
import pickle
import tables_io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from jax import numpy as jnp

from rail.core.data import TableHandle
from rail.core.stage import RailStage
from rail.utils.path_utils import RAILDIR
from rail.core.common_params import SHARED_PARAMS

from rail.shire import ShireInformer, ShireEstimator, hist_outliers, plot_zp_zs_ensemble

jax.config.update("jax_enable_x64", True)

In [ ]:
RAILDIR

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

## Select and load data into the datastore

In [ ]:
trainFile = '/global/cfs/projectdirs/desc-pz/DP1/data/train/dp1_matched_v4_train.hdf5'
testFile = '/global/cfs/projectdirs/desc-pz/DP1/data/test/dp1_matched_v4_test.hdf5'

train_dict = tables_io.read(trainFile)
test_dict = tables_io.read(testFile)

In [ ]:
train_tab = tables_io.convert(train_dict, tables_io.types.PD_DATAFRAME)
test_tab = tables_io.convert(test_dict, tables_io.types.PD_DATAFRAME)

In [ ]:
rename_euclid = {f'euclid_{eb}_unifMag': f'mag_{eb}_euclid' for eb in 'yjh'}
rename_euclid_err = {f'euclid_{eb}_unifMagErr': f'mag_err_{eb}_euclid' for eb in 'yjh'}
rename_lsst = {f'{b}_cModelMag': f'mag_{b}_lsst' for b in 'ugrizy'}
rename_lsst_err = {f'{b}_cModelMagErr': f'mag_err_{b}_lsst' for b in 'ugrizy'}

rename_dict = {**rename_euclid, **rename_euclid_err, **rename_lsst, **rename_lsst_err}
rename_dict

In [ ]:
train_tab.rename(columns=rename_dict, inplace=True)
test_tab.rename(columns=rename_dict, inplace=True)

In [ ]:
_seltrainsampl = np.logical_and(train_tab['redshift']<=3.0, train_tab['mag_i_lsst']<=25.3)
_seltraineucl = np.logical_and(
    np.isfinite(train_tab['mag_y_euclid']),
    np.logical_and(
        np.isfinite(train_tab['mag_j_euclid']),
        np.logical_and(np.isfinite(train_tab['mag_h_euclid']),
                       np.logical_and(np.isfinite(train_tab['mag_err_y_euclid']),
                                      np.logical_and(np.isfinite(train_tab['mag_err_j_euclid']),
                                                     np.isfinite(train_tab['mag_err_h_euclid'])
                                                    )
                                     )
                      )
    )
)
_seltrain = np.logical_and(_seltrainsampl, _seltraineucl)
train_tab[_seltrain]

In [ ]:
_seltestsampl = np.logical_and(test_tab['redshift']<=3.0, test_tab['mag_i_lsst']<=25.3)
_seltesteucl = np.logical_and(
    np.isfinite(test_tab['mag_y_euclid']),
    np.logical_and(
        np.isfinite(test_tab['mag_j_euclid']),
        np.logical_and(np.isfinite(test_tab['mag_h_euclid']),
                       np.logical_and(np.isfinite(test_tab['mag_err_y_euclid']),
                                      np.logical_and(np.isfinite(test_tab['mag_err_j_euclid']),
                                                     np.isfinite(test_tab['mag_err_h_euclid'])
                                                    )
                                     )
                      )
    )
)
_seltest = np.logical_and(_seltestsampl, _seltesteucl)
test_tab[_seltest]

In [ ]:
training_data = DS.add_data("training_data", train_tab[_seltrain].copy(), TableHandle)
test_data = DS.add_data(
    "test_data",
    pd.concat(
        (train_tab[_seltrain], test_tab[_seltest]),
        axis=0, join='inner', ignore_index=True
    ),
    TableHandle
)

In [ ]:
#trainFile = 'dp1_matched_v4_train_forshire.h5' #'/global/cfs/projectdirs/desc-pz/DP1/data/train/dp1_matched_v4_train.hdf5'
#testFile = 'dp1_matched_v4_train_forshire.h5' #'/global/cfs/projectdirs/desc-pz/DP1/data/test/dp1_matched_v4_test.hdf5'

#training_data = DS.read_file("training_data", TableHandle, trainFile)
#test_data = DS.read_file("test_data", TableHandle, testFile)

In [ ]:
lsst_filts_dict = {f"{_n}_lsst": "filt_lsst" for _n in "ugriz"}
euclid_filts_dict = {f"{_n}_euclid": "filt_euclid" for _n in "yjh"}

_bands = [ f"mag_{_k}" for _k in lsst_filts_dict ] + [ f"mag_{_k}" for _k in euclid_filts_dict ] 
_errbands = [ f"mag_err_{_k}" for _k in lsst_filts_dict ] + [ f"mag_err_{_k}" for _k in euclid_filts_dict ] 

In [ ]:
_bands

## Inform the estimator, i.e. select a subset of galaxies as templates

In [ ]:
default_dict_inform = dict(
    hdf5_groupname="", #False, #"photometry",
    #data_path="./data",
    bands=_bands,
    err_bands=_errbands,
    spectra_file="dsps_valid_fits_F2SM3_GG_DESI.h5", # "dsps_valid_fits_GG_DESI.h5", # 
    ssp_file="ssp_data_fsps_v3.2_lgmet_age.h5",
    filter_dict={**lsst_filts_dict, **euclid_filts_dict},
    wlmin=500.,
    wlmax=25000.,
    dwl=5.,
    #randomsel=True,
    ntemplates=50
)

### Prepare two informers: one for each method 'SPS' or 'Legacy'
- 'SPS' recomputes an SED at every redshift based on the star-formation history of the template galaxy before synthesizing the colours for every value of $z$ along the grid
- 'Legacy' computes the SED once at the template galaxy's redshift and uses it to synthesize colours at all $z$ values with the usual transformation $\lambda_\mathrm{obs} = (1+z)\lambda_\mathrm{em}$

In [ ]:
run_shire_inform_sps = ShireInformer.make_stage(
    name="shireSPS_inform_descDP1_euclid",
    output="shireSPS_templates_descDP1_euclid.h5",
    **default_dict_inform,
    templ_type="SPS"
)

run_shire_inform_legacy = ShireInformer.make_stage(
    name="shireLEG_inform_descDP1_euclid",
    output="shireLEG_templates_descDP1_euclid.h5",
    **default_dict_inform,
    templ_type="Legacy"
)

In [ ]:
test_data()

### Inform the 'SPS' templates

In [ ]:
%%time
run_shire_inform_sps.inform(training_data)

In [ ]:
templ = run_shire_inform_sps.get_handle("templates")
templ.read()

In [ ]:
templ_df = templ.read()
list_f2 = [n for n, row in templ_df.iterrows() if 'fors2' in row['Dataset'].lower()]
list_gg = [n for n, d in templ_df['Dataset'].items() if 'gogreen' in d.lower()]
list_desi = [n for n, d in templ_df['Dataset'].items() if 'desi' in d.lower()]

In [ ]:
print(json.dumps(list_f2))
print(json.dumps(list_gg))
print(json.dumps(list_desi))

In [ ]:
run_shire_inform_sps.plot_colrs_templates()

In [ ]:
run_shire_inform_sps.plot_sfh_templates()

In [ ]:
run_shire_inform_sps.hist_colrs_templates()

In [ ]:
all_templs_df_sps = run_shire_inform_sps._nuvk_classif()
f, a = plt.subplots(1,1)
sns.scatterplot(
    data=all_templs_df_sps, x="g_lsst-r_lsst", y="r_lsst-i_lsst",
    hue="CAT_NUVK", hue_order=['E_S0', 'Sbc/Scd', 'Irr'], palette='tab10', alpha=0.5,
    size='z_p', sizes=(10, 100),
    ax=a
)
a.grid()

In [ ]:
run_shire_inform_sps.plot_templ_seds()

### Inform the 'Legacy' templates

In [ ]:
%%time
run_shire_inform_legacy.inform(training_data)

In [ ]:
run_shire_inform_legacy.plot_templ_seds()

In [ ]:
run_shire_inform_legacy.plot_colrs_templates()

In [ ]:
all_templs_df_leg = run_shire_inform_legacy._nuvk_classif()
f, a = plt.subplots(1,1)
sns.scatterplot(
    data=all_templs_df_leg, x="g_lsst-r_lsst", y="r_lsst-i_lsst",
    hue="CAT_NUVK", hue_order=['E_S0', 'Sbc/Scd', 'Irr'], palette='tab10', alpha=0.5,
    size='z_p', sizes=(10, 100),
    ax=a
)
a.grid()

### Plot the prior

In [ ]:
trainref_df = pd.DataFrame()
trainref_df['ref_mag'] = np.array(run_shire_inform_sps.refmags)
trainref_df['redshift'] = np.array(run_shire_inform_sps.szs)
trainref_df['type_num'] = np.array(run_shire_inform_sps.besttypes)
trainref_df['type'] = np.array([run_shire_inform_sps.refcategs[_n] for _n in run_shire_inform_sps.besttypes])

In [ ]:
trainref_df[['u-g', 'g-r', 'r-i', 'i-z', 'z-y']] = np.array(run_shire_inform_sps.mags[:, :5] - run_shire_inform_sps.mags[:, 1:6])

In [ ]:
f,a = plt.subplots(1,1)
sns.histplot(data=trainref_df, x='ref_mag', hue='type', hue_order=['E_S0', 'Sbc/Scd', 'Irr'], palette='tab10', stat='probability', multiple='layer', ax=a)

In [ ]:
sns.scatterplot(trainref_df, x='u-g', y='i-z', hue='type', hue_order=['E_S0', 'Sbc/Scd', 'Irr'], palette='tab10', alpha=0.5)

In [ ]:
f,a = plt.subplots(1,1)
sns.histplot(data=trainref_df, x='redshift', hue='type', hue_order=['E_S0', 'Sbc/Scd', 'Irr'], palette='tab10', stat='density', multiple='layer', ax=a) 

In [ ]:
f,a = plt.subplots(1,1)
sns.kdeplot(data=trainref_df, x='ref_mag', y='redshift', hue='type', hue_order=['E_S0', 'Sbc/Scd', 'Irr'], palette='tab10', ax=a)

Check out [compare_priors.ipynb](compare_priors.ipynb) !

## Run the photometric redshifts estimation

### Build two estimators
Again, one is for the "SPS" method and the other is for the "Legacy" method. Though both `estimate` stages could work with templates from either `inform` stage, it makes more sense to keep things consistent and load the appropriate `handles` from the corresponding `inform` stage.

In [ ]:
use_prior = True
_suffix = "" if use_prior else "_noprior"

In [ ]:
default_dict_estimate = dict(
    hdf5_groupname="", #"photometry",
    #data_path="./data",
    bands=_bands,
    err_bands=_errbands,
    zmin=0.01,
    zmax=3.1,
    nzbins=510,
    ssp_file="ssp_data_fsps_v3.2_lgmet_age.h5",
    filter_dict={**lsst_filts_dict, **euclid_filts_dict},
    wlmin=500.,
    wlmax=25000.,
    dwl=5.,
    no_prior=not(use_prior),
    chunk_size=1000
)

run_shire_estimate_sps = ShireEstimator.make_stage(
    name="shireSPS_estimate_descDP1_euclid"+_suffix,
    output=f"shireSPS_results_descDP1_euclid{_suffix}.hdf5",
    **default_dict_estimate,
    templ_type="SPS",
    templates=run_shire_inform_sps.get_handle("templates"),
    model=run_shire_inform_sps.get_handle("model")
)

run_shire_estimate_legacy = ShireEstimator.make_stage(
    name="shireLEG_estimate_descDP1_euclid"+_suffix,
    output=f"shireLEG_results_descDP1_euclid{_suffix}.hdf5",
    **default_dict_estimate,
    templ_type="Legacy",
    templates=run_shire_inform_legacy.get_handle("templates"),
    model=run_shire_inform_legacy.get_handle("model")
)

### Run a reference Estimator (BPZ)

In [ ]:
typefile = 'training_types_legacy_dp1_euclid.hdf5'
typ_df = pd.DataFrame()
traintypes = np.array(run_shire_inform_sps.besttypes)
typ_df['types'] = traintypes #np.where(traintypes > 1, traintypes-1, traintypes)
tables_io.write(typ_df, typefile)
typ_df['types']

In [ ]:
from rail.estimation.algos.bpz_lite import BPZliteInformer, BPZliteEstimator

from rail.core.data import ModelHandle

RAILDIR = "/global/u2/j/jcheval/rail_base/src"

cosmospriorfile = os.path.join(RAILDIR, "rail/examples_data/estimation_data/data/COSMOS31_HDFN_prior.pkl")
cosmosprior = DS.read_file("cosmos_prior", ModelHandle, cosmospriorfile)
sedfile = "COSMOS_seds.list" #os.path.join(RAILDIR, "rail/examples_data/estimation_data/data/SED/COSMOS_seds.list")

filter_list = [f"DC2LSST_{b.split('_')[0]}" for b in lsst_filts_dict.keys()]

cosmos_dict = dict(
    hdf5_groupname="", #"photometry",
    output=f"BPZ_results_descDP1_euclid{_suffix}.hdf5",
    spectra_file=sedfile,
    bands=[ f"mag_{_k}_lsst" for _k in "ugrizy"],
    err_bands=[ f"mag_err_{_k}_lsst" for _k in "ugrizy"],
    filter_list=[f"DC2LSST_{_k}" for _k in "ugrizy"],
    wlmin=500.,
    wlmax=25000.,
    dwl=5.,
    zmin=0.01,
    zmax=3.1,
    nzbins=510,
    prior_band="mag_i_lsst",
    data_path="/global/u2/j/jcheval/rail_base/src/rail/examples_data/estimation_data/data",
    no_prior=not(use_prior),
    chunk_size=5000
)

inform_bpz = BPZliteInformer.make_stage(
    name="BPZ_inform_descDP1_euclid",
    hdf5_groupname="", #"photometry",
    nondetect_val=jnp.nan,
    bands=[ f"mag_{_k}_lsst" for _k in "ugrizy"],
    err_bands=[ f"mag_err_{_k}_lsst" for _k in "ugrizy"],
    filter_list=[f"DC2LSST_{_k}" for _k in "ugrizy"],
    prior_band="mag_i_lsst",
    wlmin=500.,
    wlmax=25000.,
    dwl=5.,
    data_path="/global/u2/j/jcheval/rail_base/src/rail/examples_data/estimation_data/data",
    nt_array=[10, 5, 16],
    type_file=typefile
)

In [ ]:
%%time
inform_bpz.inform(training_data)

In [ ]:
estimate_bpz = BPZliteEstimator.make_stage(
    name="BPZ_estimate_descDP1_euclid"+_suffix,
    model= cosmosprior, # inform_bpz.get_handle("model"), #
    **cosmos_dict
)

In [ ]:
%%time
estimate_bpz.estimate(test_data)

In [ ]:
pdfs_file_bpz = f"BPZ_results_descDP1_euclid{_suffix}.hdf5"
custom_res_bpz = qp.read(pdfs_file_bpz)
sz = jnp.array(test_data()['redshift']) #['photometry']['redshift'])

In [ ]:
a = plot_zp_zs_ensemble(
    custom_res_bpz, sz,
    z_grid=None, key_estim="zmode",
    label='_'.join(['BPZ']+(os.path.splitext(pdfs_file_bpz)[0]).split('_')[2:]))
plt.show()

### Run the "SPS" estimation

In [ ]:
%%time
run_shire_estimate_sps.estimate(test_data)

In [ ]:
pdfs_file_sps = f"shireSPS_results_descDP1_euclid{_suffix}.hdf5"
custom_res_sps = qp.read(pdfs_file_sps)
sz = jnp.array(test_data()['redshift']) #['photometry']['redshift'])

In [ ]:
a = plot_zp_zs_ensemble(
    custom_res_sps, sz,
    z_grid=None, key_estim="zmode",
    label='_'.join(['SHIRE_SPS']+(os.path.splitext(pdfs_file_sps)[0]).split('_')[2:]))
plt.show()

### Run the "Legacy" estimation

In [ ]:
%%time
run_shire_estimate_legacy.estimate(test_data)

In [ ]:
pdfs_file_legacy = f"shireLEG_results_descDP1_euclid{_suffix}.hdf5"
custom_res_legacy = qp.read(pdfs_file_legacy)
# sz = jnp.array(test_data()['photometry']['redshift']) -- Unnecessary

In [ ]:
a = plot_zp_zs_ensemble(
    custom_res_legacy, sz,
    z_grid=None, key_estim="zmode",
    label='_'.join(['SHIRE_Legacy']+(os.path.splitext(pdfs_file_legacy)[0]).split('_')[2:]))
plt.show()

### Compare outliers distribution between both methods

In [ ]:
#pdfs_file_bpz = "bpz_results_COSMOS_SEDs_LSST.hdf5"
#custom_res_bpz = qp.read(pdfs_file_bpz)

In [ ]:
hist_outliers(
    custom_res_sps, sz, label1='_'.join(['SHIRE_SPS']+(os.path.splitext(pdfs_file_sps)[0]).split('_')[2:]),
    qp_ens_2=custom_res_legacy, label2='_'.join(['SHIRE_Legacy']+(os.path.splitext(pdfs_file_legacy)[0]).split('_')[2:]),
    qp_ens_3=custom_res_bpz, label3='_'.join(['BPZ']+(os.path.splitext(pdfs_file_legacy)[0]).split('_')[2:])
)
plt.show()

## Evaluate posteriors using `RAIL` 

Check out [Evaluation_demo_LSSTsim_v2.ipynb](rail_shire/examples/Evaluation_demo_LSSTsim_v2.ipynb) !

## Build a pipeline with `ceci`

In [ ]:
import ceci
pipe = ceci.Pipeline.interactive()
stages = [run_shire_inform_sps, run_shire_estimate_sps]
for stage in stages:
    pipe.add_stage(stage)
pipe.stage_execution_config['shireSPS_estimate_descDP1_euclid'+_suffix].nprocess=1

In [ ]:
pipe.initialize(
    dict(
        training_data=trainFile,
        test_data=testFile
    ),
    dict(
        output_dir='.',
        log_dir='.',
        resume=False
    ),
    None
)

In [ ]:
pipe.save(f'rail_shireSPS_descDP1_euclid_pz{_suffix}.yml')

In [ ]:
pr = ceci.Pipeline.read(f'rail_shireSPS_descDP1_euclid_pz{_suffix}.yml')

In [ ]:
#pr.run()